In [1]:
import forecasting

import pandas as pd
from tqdm import tqdm

import datetime
import time

tickers = ['BTC/USDT','ETH/USDT','NEO/USDT','LTC/USDT',
           'ADA/USDT','XRP/USDT','IOTA/USDT','XLM/USDT',
           'ONT/USDT','TRX/USDT','ETC/USDT','VET/USDT',
           'LINK/USDT','ZEC/USDT','MATIC/USDT','DOGE/USDT']

# tickers = forecasting.get_USDT_tickers()
# tickers = tickers[:20]

In [2]:
steps = 5
delay = 60 * steps # 1m 일때

cumulative_return = 1
count = 1

while True:
    print(f'********************************************************************************** Trade {count} **********************************************************************************')
    crypto = forecasting.cryptoGenerator(crypto_list=tickers, feature='close', timeframe='1m', limit=1000) # 시간관계상 1m 으로 test
    results_df = pd.DataFrame(columns=['암호화폐', 'final_return'])

    for ticker in tickers:
        x_train, y_train = forecasting.trainDataGenerator(crypto_data=crypto, ticker=ticker, test=False)
        final_return = forecasting.cryptoNLinear(crypto, x_train, y_train, ticker=ticker, test=False)
        results_df = results_df.append({'암호화폐': ticker, 'final_return': final_return}, ignore_index=True)

    results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
    df_sorted = results_df.sort_values('순위', ascending=True)

    long_list = df_sorted[df_sorted['final_return'] > 0]['암호화폐'].to_list()
    short_list = df_sorted[df_sorted['final_return'] < 0]['암호화폐'].to_list()

    print(f'롱 포트폴리오 구성목록: {long_list}')
    print(f'숏 포트폴리오 구성목록: {short_list}')

    long_start = forecasting.cryptoGenerator(long_list, 'close', '1m', 3).tail(1)
    short_start = forecasting.cryptoGenerator(short_list, 'close', '1m', 3).tail(1)
    # print(long_start)
    # print(short_start)

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(f'[{formatted_time}] 롱, 숏 포지션 진입')

    time.sleep(delay)

    long_end = forecasting.cryptoGenerator(long_list, 'close', '1m', 3).tail(1)
    short_end = forecasting.cryptoGenerator(short_list, 'close', '1m', 3).tail(1)
    # print(long_end)
    # print(short_end)

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(f'[{formatted_time}] 롱, 숏 포지션 청산')

    long_return = (long_end.reset_index(drop=True) - long_start.reset_index(drop=True)) / long_start.reset_index(drop=True) * 100
    short_return = (short_start.reset_index(drop=True) - short_end.reset_index(drop=True)) / short_start.reset_index(drop=True) * 100
    # print(long_return)
    # print(short_return)

    
    # 투자 비율은 동일하다고 가정
    long_portfolio_return = float(long_return.sum(axis=1))
    short_portfolio_return = float(short_return.sum(axis=1))
    total_portfolio_return = long_portfolio_return + short_portfolio_return

    cumulative_return = cumulative_return * (1 + total_portfolio_return / 100)

    print(f'롱 포트폴리오 수익률: {long_portfolio_return:.3f} %')
    print(f'숏 포트폴리오 수익률: {short_portfolio_return:.3f} %')
    print(f'전체 포트폴리오 수익률: {total_portfolio_return:.3f} %')

    print(f'누적 수익률: {((cumulative_return - 1) * 100):.3f} %')

    count += 1

********************************************************************************** Trade 1 **********************************************************************************


KeyboardInterrupt: 